In [1]:
# %pip install pytesseract PyMuPDF
# %pip install tabula-py[jpype]
# %pip install python-docx

In [ ]:
# instalador do tesseract: https://github.com/UB-Mannheim/tesseract/wiki
# baixar por.traineddata https://github.com/tesseract-ocr/tessdata/blob/main/por.traineddata (leitura em português)

In [ ]:
# instalador do tesseract: https://github.com/UB-Mannheim/tesseract/wiki
# baixar por.traineddata https://github.com/tesseract-ocr/tessdata/blob/main/por.traineddata (leitura em português)

In [ ]:
# os.environ['TESSDATA_PREFIX'] = r'C:/Program Files/Tesseract-OCR/tessdata'

In [1]:
import tabula
#import PyPDF2
import os
import json
# import io
import re
import tabula
import os
import json
import fitz  # PyMuPDF
import pytesseract
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
# from openai import OpenAI
from openpyxl import load_workbook
from unidecode import unidecode
from tqdm.notebook import tqdm
from PIL import Image
from unidecode import unidecode
from tqdm.notebook import tqdm
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH


In [ ]:
os.environ['JAVA_HOME'] = 'C:/Program Files (x86)/Java/jdk-11.0.22/bin'

### TED

In [ ]:
def gera_lista_arquivos(caminho):
    arquivos = [arquivo for arquivo in os.listdir(caminho) if os.path.isfile(os.path.join(caminho, arquivo))]

    texto_estorno = 'ESTORNO'
    texto_ded = 'DED'

    sublista_e = [arquivo for arquivo in arquivos if texto_estorno in arquivo]
    sublista_d = [arquivo for arquivo in arquivos if texto_ded in arquivo]

    return sublista_e, sublista_d

In [ ]:
def captura_ted(caminho):
    dados = []
    tabula_dfs = tabula.read_pdf(caminho, pages='all', area=[1,1,3000,3000], multiple_tables='False')
    for i in range(0,len(tabula_dfs)):
        date_pattern = r'\d{2}/\d{2}/\d{4}'
        date_match = re.search(date_pattern, tabula_dfs[0]['Transferência Eletrônica Disponível - TED "IF-CLI"'].iloc[16])
        valor = tabula_dfs[i]['Unnamed: 3'].iloc[11]
        hora = tabula_dfs[i]['Unnamed: 0'].iloc[11]

        dicionario = {
            'data': date_match.group() if date_match else None,
            'hora': hora,
            'valor': valor,
            'pagina': i+1
        }

        dados.append(dicionario)

    df = pd.DataFrame(dados)
    return df

### DED

In [ ]:
def captura_ded(caminho):    
    tabula_dfs = tabula.read_pdf(caminho, pages='all', area=[225,1,3000,3000], multiple_tables='True')

    tabela_concat = pd.concat(tabula_dfs, ignore_index=True)[["DT. VCTO.","PARC.","VLR. FINAL","TOT. DIVIDA","VLR.LNC./PG.","HIST. FINANC."]].dropna(subset=["DT. VCTO."])
    tabela_concat = tabela_concat[tabela_concat["DT. VCTO."] != "Continua ..."]
    tabela_concat[['codigo', 'descricao']] = tabela_concat['HIST. FINANC.'].str.split(' - ', expand=True)
    tabela_concat['valor'] = tabela_concat['VLR.LNC./PG.'].str.extract(r'(\d{1,3}(?:\.\d{3})*,\d{2})')
    df = tabela_concat[tabela_concat['codigo'] == '101'][['DT. VCTO.','PARC.','valor']].reset_index(drop=True)

    return df

In [ ]:
def varios_deds(sublista_d,caminho):
    lista_df_deds = []

    for i in range(len(sublista_d)):
        df_intermediario = captura_ded(f'{caminho}/{sublista_d[i]}')

        lista_df_deds.append(df_intermediario)

    for df in lista_df_deds:
        df['data_ref'] = pd.to_datetime(df['DT. VCTO.'], dayfirst=True)
        df['data_ref'] = df['data_ref'].apply(lambda x: x.replace(day = 1) + pd.offsets.MonthEnd(1))
        df['valor'] = df['valor'].str.replace('.', '').str.replace(',', '.').astype(float)
        df.set_index('data_ref', inplace=True)

    if len(lista_df_deds) == 0:
        df_agrupado_soma = df
    if len(lista_df_deds) >= 0:
        df_concatenado = pd.concat(lista_df_deds)

        df_agrupado_soma = df_concatenado.groupby(df_concatenado.index).agg({'valor': 'sum'})
        df_agrupado_soma.reset_index(inplace=True)
    
    return df_agrupado_soma

### Input Excel

In [ ]:
erros = []
def gerar_excel(folder_especifico,dic_parametros):
    sublista_estorno, sublista_ded = gera_lista_arquivos(folder_especifico)

    if len(sublista_estorno) > 0:
        try:
            df_ted = captura_ted(f'{folder_especifico}/{sublista_estorno[0]}')
        except:
            dic = {
                'caminho': folder_especifico,
                'lista': sublista_estorno
            }
            erros.append(dic)

    if len(sublista_ded) > 0:
        try:
            df_ded = varios_deds(sublista_ded,folder_especifico)
        except:
            dic = {
                'caminho': folder_especifico,
                'lista': sublista_ded
            }
            erros.append(dic)

    ##### Excel
    wb = load_workbook('base.xlsx')

    sheet_sumario = wb['Sumário']
    dic_parametros = dic_parametros.iloc[0]
    processo = dic_parametros['n_processo']

    sheet_sumario['H1'].value = dic_parametros['base_atualizacao']
    sheet_sumario['C2'].value = dic_parametros['n_processo']
    sheet_sumario['C3'].value = dic_parametros['autor']
    sheet_sumario['C4'].value = dic_parametros['data_distribuicao']
    sheet_sumario['C5'].value = dic_parametros['citacao']
    sheet_sumario['C6'].value = dic_parametros['valor_causa']

    sheet_sumario['C8'].value = dic_parametros['dano_moral']
    sheet_sumario['D8'].value = dic_parametros['dano_moral_data_cor']
    sheet_sumario['E8'].value = dic_parametros['dano_moral_data_jur']
    sheet_sumario['G8'].value = dic_parametros['dano_moral_indice']

    sheet_sumario['C9'].value = dic_parametros['honorario_sucumbencial']
    sheet_sumario['D9'].value = dic_parametros['honorario_sucumbencial_data_cor']
    sheet_sumario['E9'].value = dic_parametros['honorario_sucumbencial_data_jur']
    sheet_sumario['G9'].value = dic_parametros['honorario_sucumbencial_indice']

    sheet_sumario['C10'].value = dic_parametros['outras_multas']
    sheet_sumario['D10'].value = dic_parametros['outras_multas_data_cor']
    sheet_sumario['E10'].value = dic_parametros['outras_multas_data_jur']
    sheet_sumario['G10'].value = dic_parametros['outras_multas_indice']

    sheet_sumario['C11'].value = dic_parametros['multa_475']
    sheet_sumario['D11'].value = dic_parametros['multa_475_data_cor']
    sheet_sumario['E11'].value = dic_parametros['multa_475_data_jur']
    sheet_sumario['G11'].value = dic_parametros['multa_475_indice']

    sheet_sumario['C12'].value = dic_parametros['multa_ma_fe']
    sheet_sumario['D12'].value = dic_parametros['multa_ma_fe_data_cor']
    sheet_sumario['E12'].value = dic_parametros['multa_ma_fe_data_jur']
    sheet_sumario['G12'].value = dic_parametros['multa_ma_fe_indice']

    sheet_sumario['C13'].value = dic_parametros['outros']
    sheet_sumario['D13'].value = dic_parametros['outros_data_cor']
    sheet_sumario['E13'].value = dic_parametros['outros_data_jur']
    sheet_sumario['G13'].value = dic_parametros['outros_indice']

    sheet_sumario['C14'].value = dic_parametros['empr_n_devol']
    sheet_sumario['D14'].value = dic_parametros['empr_n_devol_data_cor']
    sheet_sumario['E14'].value = dic_parametros['empr_n_devol_data_jur']
    sheet_sumario['G14'].value = dic_parametros['empr_n_devol_indice']

    sheet_sumario['C17'].value = dic_parametros['data_dep_autor']
    sheet_sumario['C18'].value = dic_parametros['valor_dep_autor']
    sheet_sumario['C21'].value = dic_parametros['data_dep_reu']
    sheet_sumario['C22'].value = dic_parametros['valor_dep_reu']


    ### PARCELAS ###
    sheet_parcelas = wb['Parcelas']

    sheet_parcelas['B2'].value = dic_parametros['p_citacao']
    sheet_parcelas['B4'].value = dic_parametros['p_indice']
    sheet_parcelas['B5'].value = dic_parametros['p_dobro']

    if 'df_ded' in locals() or 'df_ded' in globals():
    ###### DED's em formato texto, preenchimento correto.
        sheet_parcelas['B6'].value = len(df_ded)
        
        for i in range(len(df_ded)):
            sheet_parcelas[f'L{i+10}'].value = df_ded['data_ref'][i]
            sheet_parcelas[f'B{i+10}'].value = i+1
            sheet_parcelas[f'C{i+10}'].value = df_ded['valor'][i]
    
    elif dic_parametros['in_ded_texto'] == 'X':
    ###### DED's em formato de imagem, usa os TED's com base na data preenchida.
        data_inicial = dic_parametros['data_inicio_ded']

        sheet_parcelas['B6'].value = len(df_ted)
        for i in range(len(df_ted)):
            if i == 0:
                ultimo_dia_mes = data_inicial + relativedelta(day=31)
                sheet_parcelas[f'L{i+10}'].value = ultimo_dia_mes
            else:
                data_futuro = data_inicial + relativedelta(months=i, day=31)
                sheet_parcelas[f'L{i+10}'].value = data_futuro
            sheet_parcelas[f'B{i+10}'].value = i+1
            sheet_parcelas[f'C{i+10}'].value = df_ted['valor'][i]

    ## ESTORNO ###
    sheet_estorno = wb['Estorno']

    sheet_estorno['B2'].value = dic_parametros['e_citacao']
    sheet_estorno['B4'].value = dic_parametros['e_indice']
    sheet_estorno['B5'].value = dic_parametros['e_dobro']

    if 'df_ted' in locals() or 'df_ted' in globals():
        sheet_estorno['B2'].value = df_ted['data'][0]
        sheet_estorno['B6'].value = len(df_ted)

        for i in range(len(df_ted)):
            sheet_estorno[f'L{i+10}'].value = df_ted['data'][i]
            sheet_estorno[f'B{i+10}'].value = i+1
            sheet_estorno[f'C{i+10}'].value = df_ted['valor'][i]

    wb.save(f'{folder_especifico}/calculo_{processo}.xlsx')

In [ ]:
dados_processos = pd.read_excel('dados_processos.xlsx')

In [ ]:
folder = 'C:/Users/victor.serra/Desktop/python/automacao/projeto_derroted/processos'

pastas = [nome for nome in os.listdir(folder) if os.path.isdir(os.path.join(folder, nome))]

In [ ]:
erros = []
for i in range(len(pastas)):
    pasta_agora = pastas[i]
    filtro = dados_processos['pasta'] == pasta_agora
    dic_parametros = dados_processos[filtro]
    folder_especifico = f'{folder}/{pasta_agora}'

    gerar_excel(folder_especifico,dic_parametros)

In [ ]:
pd.DataFrame(erros).to_excel('verificar.xlsx')

In [16]:
modelo_laudo = Document('laudo_base.docx')

In [20]:
# Informações Básicas
modelo_laudo.tables[0].rows[2].cells[1].paragraphs[0].text = 'num_process_auto'
modelo_laudo.tables[0].rows[3].cells[1].paragraphs[0].text = 'num_processo'
modelo_laudo.tables[0].rows[4].cells[1].paragraphs[0].text = 'nome_autor'
modelo_laudo.tables[0].rows[5].cells[1].paragraphs[0].text = 'estrategia'
modelo_laudo.tables[0].rows[6].cells[1].paragraphs[0].text = 'adv_monitorado'
modelo_laudo.tables[0].rows[7].cells[1].paragraphs[0].text = 'data_distribuicao'
modelo_laudo.tables[0].rows[8].cells[1].paragraphs[0].text = 'data_citacao'
modelo_laudo.tables[0].rows[9].cells[1].paragraphs[0].text = 'valor_acao'
modelo_laudo.tables[0].rows[10].cells[1].paragraphs[0].text = 'revelia'
modelo_laudo.tables[0].rows[11].cells[1].paragraphs[0].text = 'modalidade_pgto'

# Tipo pgto
modelo_laudo.tables[0].rows[12].cells[1].paragraphs[0].text = 'Principal' # negrito
modelo_laudo.tables[0].rows[12].cells[1].paragraphs[1].text = f"Tipo: {'Condenação'}"

# Sentença
modelo_laudo.tables[0].rows[14].cells[1].paragraphs[0].text = 'Sim'          # col excel. SIM | NÃO
modelo_laudo.tables[0].rows[14].cells[1].paragraphs[1].text = f"Resultado: {'Parcial Procedente'}"
modelo_laudo.tables[0].rows[14].cells[1].paragraphs[2].text = f"Data Publicação: {'22/09/2023'}"
# Dispositivo Sentença
modelo_laudo.tables[0].rows[16].cells[0].paragraphs[0].text = 'Sentença'

# Recurso:
modelo_laudo.tables[0].rows[17].cells[1].paragraphs[0].text = 'Sim'          # col excel. SIM | NÃO
modelo_laudo.tables[0].rows[17].cells[1].paragraphs[1].text = f"Parte(s) Ingressante(s): {' Ambos | Autor | Réu '}"
modelo_laudo.tables[0].rows[17].cells[1].paragraphs[2].text = f"Resultado: {'Parcial Procedente'}"
modelo_laudo.tables[0].rows[17].cells[1].paragraphs[3].text = f"Data Publicação: {'22/09/2023'}"
# Dispositivo Recurso
modelo_laudo.tables[0].rows[19].cells[0].paragraphs[0].text = 'Dispositivo_recurso'
    # {'recurso': "Sim | Não ", Paragrafo, ,Paragrafo, "Resultado": {lista_res}, Paragrafo, 'data_publicacao': {data_publicacao}}



In [22]:
linhas_formatacao_base = [2,3,4,5,6,7,8,9,10,11,12,14,16,17,19]
linhas_negrito = [[12,0],[14,0],[17,0]]

for i in linhas_formatacao_base:
    base = modelo_laudo.tables[0].rows[i].cells[1]
    for y in range(len(base.paragraphs)):
        run = base.paragraphs[y].runs[0]
        run.font.name = 'Montserrat'
        run.font.size = Pt(9)

for linha in linhas_negrito:
    base = modelo_laudo.tables[0].rows[int(linha[0])].cells[1].paragraphs[int(linha[1])]
    base.runs[0].font.bold = True

In [ ]:
# modelo_laudo.tables[0].rows[19].cells[0].text - dispositivo_recurso
# modelo_laudo.tables[0].rows[20].cells[1].text - OBF
    # {'forma': "{Sim | Nao}", Paragrafo, 'Status': '{lista_status}', Paragrafo, "Descrever": ...}
# modelo_laudo.tables[0].rows[22].cells[1].text - dano_material
    # {'forma': "{sim | nao}", Paragrafo, valor, valor_atualizado, data_correcao, data_juros, forma: {"lista"}}
# modelo_laudo.tables[0].rows[23].cells[1].text - dano_moral
    # {'forma': "{sim | nao}", Paragrafo, valor, valor_atualizado, data_correcao, data_juros, forma: {"lista"}}
# modelo_laudo.tables[0].rows[24].cells[1].text - hon_sucumbenciais
    # {forma: "{sim | nao}", valor}
# modelo_laudo.tables[0].rows[25].cells[1].text - multa_outros
    # {forma: "{sim | nao}", valor}
# modelo_laudo.tables[0].rows[26].cells[1].text - multa_475_J
    # {forma: "{sim | nao}", valor}
# modelo_laudo.tables[0].rows[27].cells[1].text - multa_lit_ma_fe
    # {forma: "{sim | nao}", valor}
# modelo_laudo.tables[0].rows[28].cells[1].text - multa_obf
    # {forma: "{sim | nao}", valor}
# modelo_laudo.tables[0].rows[29].cells[1].text - outros_valores
    # {forma: "{sim | nao}", motivo, valor}
# modelo_laudo.tables[0].rows[31].cells[0].text - calculo
    # imagem
# modelo_laudo.tables[0].rows[32].cells[1].text - valor_total
    # {forma: valor}
# modelo_laudo.tables[0].rows[33].cells[1].text - estorno_parcelas
    # {forma: "{sim | nao}", valor}
# modelo_laudo.tables[0].rows[34].cells[1].text - valor_compensacao
    # {forma: "{sim | nao}", valor}
# modelo_laudo.tables[0].rows[35].cells[1].text - total_a_pagar
    # {valor}
# modelo_laudo.tables[0].rows[36].cells[1].text - tota_a_receber
    # valor


In [23]:
modelo_laudo.save('laudo_teste.docx')

In [ ]:
# pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'  # Caminho para o executável do tesseract

# parse_pdf = []

# Abra o arquivo PDF com o PyMuPDF
# pdf_file = f'{folder_especifico}/{sublista_ded[0]}'
# pdf = fitz.open(pdf_file)

# Itere pelas páginas e aplique OCR
# for page_num in tqdm(range(len(pdf)), desc='Parsing pages'):
#     Obtenha a página
#     page = pdf[page_num]

#     Renderize a página em uma imagem
#     pix = page.get_pixmap()
#     img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

#     Extraia o texto da imagem usando OCR
#     content = pytesseract.image_to_string(img, lang='por')  # Especifique o idioma com o parâmetro lang, se necessário

#     dicionario = {
#         'page_num': page_num + 1,
#         'conteudo': content
#     }

#     Faça o que você precisar com o texto
#     parse_pdf.append(content)

# pdf.close()